In [16]:
import sys
sys.path.append("../../pkgs")
import pandas as pd
import numpy as np
from sksurv.metrics import concordance_index_censored,concordance_index_ipcw
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sksurv.datasets import get_x_y
from sksurv.linear_model import CoxPHSurvivalAnalysis, CoxnetSurvivalAnalysis
import joblib
import pickle

# from util.setting import DEFAULT_COL_DTYPE, DEFAULT_LABELS, DEFAULT_CONT_FEATURES, DEFAULT_FEATURES, BEST_EXTEND_FEATURES
# from util.data import standard_transform
# from util.eval import risk_cindex, eval_cindex

In [17]:
clinical_model = joblib.load('./result/OP_post_2year_coxnet_model_advance.joblib')
clinical_features = clinical_model.feature_names_in_
print(clinical_features)

['Tumor size' 'lnAST' 'BCLC' 'Tumor number' 'ALBIgrade'
 'class_Histologic grade' 'class_AFP' 'Ishak' 'Steatosis grade' 'K'
 'class_Steatosis grade' 'AST' 'HBsAg' 'Height' 'MVI']


In [18]:
train = pd.read_csv('./data/post_early/HCC OP 20230511 train_early_input_derived features.csv')
test = pd.read_csv('./data/post_early/HCC OP 20230511 test_early_input_derived features.csv')
train_std = pd.read_csv('./data/post_early/HCC OP 20230511 train_early_input_derived features_std.csv')
test_std = pd.read_csv('./data/post_early/HCC OP 20230511 test_early_input_derived features_std.csv')

In [19]:
# selected = ['Tumor size', 'ln_AST' ,'BCLC', 'Tumor number', 'ALBIgrade',
#  'class_Histologic grade', 'class_AFP', 'Ishak', 'Steatosis grade', 'K',
#  'class_Steatosis grade' ,'AST', 'HBsAg','Height','MVI',]
selected = ['Tumor size', 'ln_AST' , 'BCLC','Tumor number', 'ALBIgrade',
 'class_Histologic grade', 'class_AFP', 'Ishak','Steatosis grade', 'K' ,'class_Steatosis grade','MVI','BMI'
 ,'AST','HBsAg',]

In [20]:
train_X = train[ selected ]
train_y = train_std[['Early recur','Early RFS']].to_records(index=False).astype([('Status', 'bool'), ('Survival', 'float64')])

In [21]:
model = Pipeline([('scaler', StandardScaler()), ('cox', CoxnetSurvivalAnalysis(alphas=clinical_model.alphas, l1_ratio=clinical_model.l1_ratio, fit_baseline_model=True))])

In [22]:
print(len(train_X),len(train_y))

1033 1033


In [23]:
model.fit(train_X, train_y)

Pipeline(steps=[('scaler', StandardScaler()),
                ('cox', CoxnetSurvivalAnalysis(fit_baseline_model=True))])

In [ ]:
ibcgas_cindex = eval_cindex(model, train_X[ibcga_features], train_y, test_X[ibcga_features], test_y)

In [24]:
preds_test = model.predict(test[ selected ])
concordance_index_censored(test_std['Early recur'] > 0, test_std['Early RFS'],preds_test)

(0.7308539749102669, 41131, 15147, 0, 5)

In [25]:
joblib.dump(model, "./result/OP_post_2year_coxnet_model_advance_piplines.bin")

['./result/OP_post_2year_coxnet_model_advance_piplines.bin']